<img src="qiskit_logo.png">

# Basis Gate Decomposition of Arbitrary Two-Qubit Unitary Operators


## Contributors
Bruno Murta, Quantalab, International Iberian Nanotechnology Laboratory (INL)

## QISKit Package Versions

In [1]:
from qiskit import *
import numpy as np, cmath, sys
from numpy import linalg as LA
from qiskit.quantum_info.synthesis import two_qubit_cnot_decompose
qiskit.__qiskit_version__

{'qiskit-terra': '0.9.0',
 'qiskit-ignis': '0.2.0',
 'qiskit-aqua': '0.6.0',
 'qiskit': '0.12.0',
 'qiskit-aer': '0.3.0',
 'qiskit-ibmq-provider': '0.3.2'}

## Introduction

It is known that any n-qubit quantum computation can be expressed in terms of a sequence of single- and two-qubit quantum logic gates [1]. Nevertheless, a quantum algorithm may include operations that act on multiple qubits at once.

However, in quantum simulation, it is usually the case that all elementary operations either act on a single qubit or on two qubits at a time. For example, in quantum chemistry, one considers only single-particle and two-particle excitations to determine the most stable configuration of molecules [2]. Hence, for most practical purposes, we only really need to consider 2x2 and 4x4 unitary matrices, possibly having to perform a Trotter-Suzuki expansion [3,4] to address the presence of non-commuting terms.

Having considered 2x2 unitary matrices in the previous notebook, here we find the basis gate decomposition of any 4x4 unitary matrix and its controlled version.

## KAK Decomposition

The KAK method [5] is usually the basis for most two-qubit decomposition schemes. To explain it, we will follow the method originally proposed by Vatan and Williams [6], but following the outline given in [7].

Step 1: Transform given matrix $U$ to magic basis of Bell states: $U' = \mathcal{M}^{\dagger} U \mathcal{M}$, where $\mathcal{M}$ is the so-called _magic matrix_:

$$
\begin{split}
    & \mathcal{M} = 
    \frac{1}{\sqrt{2}} \; \left( \begin{matrix} 1 & i & 0 & 0 \\ 0 & 0 & i & 1 \\ 0 & 0 & i & -1 \\ 1 & -i & 0 & 0 \end{matrix} \right) 
\end{split}
$$

Step 2: Compute $U'^{T} U' \equiv M^2$ and diagonalize it: $M^2 = P D P^{\dagger}$

Step 3: Find $D^{\frac{1}{2}}$ and hence $K' = U' P D^{-\frac{1}{2}} P^{\dagger}$ and $A \equiv \mathcal{M} D^{\frac{1}{2}} \mathcal{M}^{\dagger}$

Step 4: It can be checked that $K'P, \; P^{\dagger} \in SO(4)$, so $K_1 \equiv \mathcal{M} K' P \mathcal{M}^{\dagger}, \; K_2 \equiv \mathcal{M} P^{\dagger} \mathcal{M}^{\dagger} \in SU(2) \times SU(2)$ (which is, in fact, the property that justifies the name of $\mathcal{M}$). Hence, $K_1$ and $K_2$ can be written as the tensor product of two single-qubit gates, i.e. $K_i = B_i \otimes C_i, \; i \in \{ 1, 2\}$. To find $B_i$ and $C_i$, take the partial trace of $K_i$, i.e. $B_i = \frac{1}{\sqrt{|\textrm{Tr}(K_i)|}} \textrm{Tr}_{2}(K_i)$ and $C_i = \frac{1}{\sqrt{|\textrm{Tr}(K_i)|}} \textrm{Tr}_{1}(K_i)$. $B_i$ and $C_i$ can then be obtained from the $U_3$ gate, as described in the previous notebook

Step 5: $A \equiv \mathcal{M} D^{\frac{1}{2}} \mathcal{M}^{\dagger}$ can be straightforwardly implemented in quantum hardware. For the case where the unitary matrix is in the Weyl chamber [6], for example, the following circuit can be used (disregarding a global phase of $\frac{\pi}{4}$). 

<img src="fig55.png" alt="Drawing" style="width: 800px;"/>

where the parameters $\alpha$, $\beta$ and $\gamma$ are related to the diagonal entries of $D^{\frac{1}{2}} = diag(e^{i(\alpha - \beta + \gamma)}, e^{-i(\alpha - \beta - \gamma)}, e^{i(\alpha + \beta - \gamma)}, e^{-i(\alpha + \beta + \gamma)})$.

Step 6: Putting all the pieces together, $U = K_1 A K_2$, which is the so-called KAK decomposition [5]

## Implementation of Arbitrary 4x4 Unitary Matrix

There is already a QISKit built-in function that implements arbitrary 4x4 unitary matrices: *qiskit.quantum_info.synthesis.two_qubit_cnot_decompose*. This function expands a given 4x4 unitary matrix into an equivalent decomposition over SU(2) + CNOT. The minimum number of CNOTs required to implement any 4x4 unitary matrix is 3 [6], but it is possible to reduce it to 2 or even 1 without significant loss in fidelity, as was recently discussed in Appendix B of [8].

Let us now exemplify the application of *two_qubit_cnot_decompose*:

In [2]:
U = qiskit.quantum_info.random_unitary(4).data
qc = two_qubit_cnot_decompose(U)

simulator = qiskit.Aer.get_backend('unitary_simulator')
result = execute(qc, simulator).result()
unitary = result.get_unitary(qc)

numerator = (np.trace(np.matmul(np.conjugate(np.transpose(U)), unitary)))**2
denominator = np.trace(np.matmul(np.conjugate(np.transpose(U)), U)) * np.trace(np.matmul(np.conjugate(np.transpose(unitary)), unitary))
fidelity_measure = numerator / denominator
print(abs(fidelity_measure))

1.0


## Controlled Two-Qubit Gates Via Diagonalization

We are now going to implement a general method to find the quantum circuit that implements any controlled two-qubit gate. This method makes use of the QISKit built-in function *two_qubit_cnot_decompose* and takes advantage of a scheme proposed by Schuch and Siewert [9] that implements controlled phase gates.

Given a two-qubit gate such that $U = V D V^{\dagger}$, with $V V^{\dagger} = I$ and $D = diag(e^{-i\theta_1}, e^{-i\theta_2}, e^{-i\theta_3}, e^{-i\theta_4})$, the controlled-U gate can be obtained as:

<img src="fig22.png" alt="Drawing" style="width: 400px;"/>

Any 4x4 unitary diagonal matrix can be written (up to a global phase) as [9]:

<img src="fig33_new.png" alt="Drawing" style="width: 600px;"/>

where $\beta$, $\gamma$ and $\delta$ are related to the diagonal entries of $D$ by:

$$
\begin{split}
    & \left( \begin{matrix} \alpha \\ \beta \\ \gamma \\ \delta \end{matrix} \right) = 
    \frac{1}{2} \; \left( \begin{matrix} 1 & 1 & 1 & 1 \\ 1 & -1 & 1 & -1 \\ 1 & 1 & -1 & -1 \\ 1 & -1 & -1 & 1 \end{matrix} \right) 
    \left( \begin{matrix} \theta_1 \\ \theta_2 \\ \theta_3 \\ \theta_4 \end{matrix} \right)
\end{split}
$$

$\alpha$ is just the global phase that was discarded in the implementation of $D$. However, it must be taken into account to obtain the correct $cD$. the The controlled-D gate is obtained by replacing all $R_z$ by $cR_z$ and adding a phase shift set by the global phase $\alpha$:

<img src="fig44_new.png" alt="Drawing" style="width: 700px;"/>

By diagonalizing the given 4x4 unitary matrix $U = V D V^{\dagger}$, finding the basis gate decomposition of the diagonalization transformations $V$ and $V^{\dagger}$ via the QISKit built-in function *two_qubit_cnot_decompose* and deriving the circuit for the controlled version of the diagonal matrix $D$ using the method described above, we can obtain the circuit for an arbitrary controlled two-qubit gate:

In [3]:
def is_unitary(A):
    '''
    Function that determines if A is a unitary matrix
    U: Unitary matrix of any size [numpy.ndarray]
    '''
    
    return np.allclose(np.eye(A.shape[0]), A.dot(np.transpose(np.conjugate(A))))

def controlled_matrix(U):
    '''
    Function that finds 8x8 matrix that represents controlled-U
    U: 4x4 unitary matrix [numpy.ndarray]
    '''
    
    top_cU = np.hstack((np.eye(4), np.zeros([4,4])))
    bottom_cU = np.hstack((np.zeros([4,4]),U))
    cU = np.vstack((top_cU,bottom_cU))
    
    return cU

def controlled_two_qubit_gate(U):
    '''
    Function that outputs quantum circuit that represents controlled-U
    U: 4x4 unitary matrix [numpy.ndarray]
    '''
    
    # Checking if input is numpy.ndarray
    if not type(U) is np.ndarray:
        sys.exit("Input must be numpy.ndarray")
    
    # Checking if input has right dimensionality
    if not U.shape == (4,4):
        sys.exit("Input must be 4x4 matrix")
        
    # Checking if input is unitary
    if not is_unitary(U):
        sys.exit("Input must be unitary")
    
    w, v = LA.eig(U)
    D = np.matmul(np.matmul(v.conj().T, U),v)

    # First, we have to decompose the diagonal matrix D in terms Rz matrices
    # The phases of the Rz matrices are given by eq. (2) from [9]
    hadamard_unnormalized = np.array([[1,1],[1,-1]])
    trans = np.kron(hadamard_unnormalized, hadamard_unnormalized)
    inv_trans = LA.inv(trans)
    input_vector = []
    for i in range(len(w)):
        input_vector.append(-2*np.angle(w[i]))
    phase_vector = np.matmul(inv_trans, input_vector)

    # Then, we must now decompose v.conj().T and v in terms of basis gates
    # This can be accomplished via the two_qubit_cnot_decompose function
    qc_v_dagger = two_qubit_cnot_decompose(v.conj().T)
    gate_v_dagger = qc_v_dagger.to_instruction()

    qc_v = two_qubit_cnot_decompose(v)
    gate_v = qc_v.to_instruction()

    # Creating a Quantum Register with 3 qubits
    qr = QuantumRegister(3)
    # Creating Quantum Circuit
    qc = QuantumCircuit(qr)

    # Controlled version of diagonal matrix simply requires replacing Rz for cRz
    # and adding the phase shift set by the global phase alpha
    qc.append(gate_v_dagger, qr[0:2])
    qc.crz(phase_vector[1], qr[2], qr[0])
    qc.crz(phase_vector[2], qr[2], qr[1])
    qc.cx(qr[1], qr[0])
    qc.crz(phase_vector[3], qr[2], qr[0])
    qc.cx(qr[1], qr[0])
    qc.rz(-phase_vector[0]/2, qr[2])
    qc.append(gate_v, qr[0:2])

    # Select the UnitarySimulator from the Aer provider
    simulator = qiskit.Aer.get_backend('unitary_simulator')
    # Execute and get counts
    result = execute(qc, simulator).result()
    unitary = result.get_unitary(qc)
    
    cU = controlled_matrix(U)
    fid_thr = 1e-6

    numerator = (np.trace(np.matmul(np.conjugate(np.transpose(cU)), unitary)))**2
    denominator = np.trace(np.matmul(np.conjugate(np.transpose(cU)), cU)) * np.trace(np.matmul(np.conjugate(np.transpose(unitary)), unitary))
    fidelity_measure = abs(numerator / denominator)
    
    if 1 - fidelity_measure < fid_thr:
        return qc
    else:
        sys.exit("Could not obtain controlled-U to desired accuracy")

Let us now test this function for an arbitrary input 4x4 unitary matrix:

In [4]:
U = qiskit.quantum_info.random_unitary(4).data
cU = controlled_matrix(U)
qc = controlled_two_qubit_gate(U)

# Select the UnitarySimulator from the Aer provider
simulator = qiskit.Aer.get_backend('unitary_simulator')
# Execute and get counts
result = execute(qc, simulator).result()
unitary = result.get_unitary(qc)
    
numerator = (np.trace(np.matmul(np.conjugate(np.transpose(cU)), unitary)))**2
denominator = np.trace(np.matmul(np.conjugate(np.transpose(cU)), cU)) * np.trace(np.matmul(np.conjugate(np.transpose(unitary)), unitary))
fidelity_measure = abs(numerator / denominator)

print(fidelity_measure)

1.0


## References

[1] A. Barenco _et al._, _Elementary gates for quantum computation_. [Phys. Rev. A 52, 3457 (1995)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.3457) <br>
[2] S. McArdle, S. Endo, A. Aspuru-Guzik, S. Benjamin, and X. Yuan, _Quantum Computational Chemistry_. [arXiv:1808.10402](https://arxiv.org/abs/1808.10402) <br>
[3] H. F. Trotter, _On the Product of Semi-Groups of Operators_. [Proc. Amer. Math. Soc. 10, 4 (1959)](https://www.jstor.org/stable/2033649?seq=1#page_scan_tab_contents) <br>
[4]   M. Suzuki, _Fractal decomposition of exponential operators with applications to many-body theories and Monte Carlo simulations_. [Phys. Lett. A 146, 319 (1990)](https://www.sciencedirect.com/science/article/abs/pii/037596019090962N) <br>
[5] R. R. Tucci, _An Introduction to Cartan's KAK Decomposition for QC Programmers_. [arXiv:quant-ph/0507171](https://arxiv.org/abs/quant-ph/0507171) <br>
[6] F. Vatan and C. Williams, _Optimal quantum circuits for general two-qubit gates_. [Phys. Rev. A 69, 032315 (2004)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.032315) <br>
[7] B. Drury and P. Love, _Constructive Quantum Shannon Decomposition from Cartan Involutions_. [J. Phys. A Math Theor. 41, 39 (2008)](https://iopscience.iop.org/article/10.1088/1751-8113/41/39/395305/meta) <br>
[8] A. W. Cross _et al._, _Validating quantum computers using randomized model circuits_. [Phys. Rev. A 100, 032328 (2019)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.100.032328) <br>
[9] N. Schuch and J. Siewert, _Programmable networks for quantum algorithms_. [Phys. Rev. Lett. 91, 027902 (2003)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.91.027902) <br>